In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import re
import statsmodels.api as sm
import scipy.stats
import calendar

### Q1 obtain risk premium of each firm characteristics using ols

In [142]:
def import_data(file):
    data = pd.read_csv(file)
    data['Return'] = data['Return'] /100
    data['Date'] = pd.to_datetime(data['Date']).apply(lambda x:x.strftime('%Y-%m'))
    # drop the months that have less 30 company
    month_group = data.groupby('Date')['GVKEY'].count() # groupby Date to count the number of stock
    # delet Date of which the number of stocks is less than 30
    less30 = month_group[month_group<30].index 
    data = data.set_index('Date').drop(less30,axis=0).reset_index()
    return data

In [143]:
def cross_section1(df): # return risk premium
    y = df['Return']
    x = df.iloc[:,3:] # treat them as beta
    x = sm.add_constant(x) 
    result = sm.OLS(y, x).fit() # OLS with constant
    return result.params

In [144]:
def cross_section2(df): # return adj R2
    y = df['Return']
    x = df.iloc[:,3:]
    x = sm.add_constant(x)
    result = sm.OLS(y, x).fit()
    return result.rsquared_adj

### Model 1

In [145]:
m1_data = import_data('GPEX1set1.csv')
m1_data

,Date,GVKEY,Return,LogSize_-1,LogB/M_-1,"Return_-2,-12"
0,1972-04,1000,0.266667,2.819480,-0.266121,-0.461539
1,1972-05,1000,-0.070175,3.053069,-0.934455,-0.476744
2,1972-06,1000,-0.094340,2.977503,-0.861695,-0.173913
3,1972-07,1000,0.000000,2.875596,-0.762604,-0.196970
4,1972-08,1000,-0.020833,2.875935,-0.762604,-0.020408
...,...,...,...,...,...,...
2290342,2020-08,328795,0.096400,7.620326,-0.130132,0.129600
2290343,2020-09,328795,-0.047526,7.712358,-0.222164,0.305948
2290344,2020-10,328795,0.048310,7.663665,-0.173471,0.359853
2290345,2020-11,328795,0.123890,7.709071,-0.219568,0.152096


In [146]:
m1_result = m1_data.groupby('Date').apply(cross_section1)
r2 = m1_data.groupby('Date').apply(cross_section2)
obs = m1_data.groupby('Date')['GVKEY'].count()
m1_result['adj_R2'] = r2
m1_result['Number_of_Firms'] = obs
m1_result.head(2)

,const,LogSize_-1,LogB/M_-1,"Return_-2,-12",adj_R2,Number_of_Firms
Date,,,,,,
1967-04,-0.007385,-0.000439,0.004839,0.127069,0.260407,39
1967-05,-0.036995,0.001527,-0.007863,-0.146485,0.169260,41


In [147]:
m1_result.tail(20)

,const,LogSize_-1,LogB/M_-1,"Return_-2,-12",adj_R2,Number_of_Firms
Date,,,,,,
2019-08,-0.036783,-0.001712,-0.001916,0.053777,0.015879,3702
2019-09,-0.013596,0.008546,0.030158,-0.038796,0.067908,3685
2019-10,-0.019680,0.004526,-0.000637,0.029990,0.015054,3612
2019-11,-0.002562,0.003886,-0.013128,-0.036935,0.008890,3629
2019-12,0.064604,-0.001202,0.017621,-0.020152,0.013839,3614
2020-01,0.061434,-0.015367,-0.027279,-0.000291,0.022052,3577
2020-02,-0.033240,-0.007448,-0.005352,-0.007484,0.004634,3564
2020-03,-0.252737,-0.002991,-0.050584,0.005838,0.046268,3546
2020-04,0.256116,-0.013799,-0.010500,-0.037973,0.016429,3696


### Model 2

In [148]:
m2_data = import_data('GPEX1set2.csv')
m2_data

,Date,GVKEY,Return,LogSize_-1,LogB/M_-1,"Return_-2,-12","LogIssues_-1,-36",Accruals_Yr-1,ROA_Yr-1,LogAG_Yr-1
0,1974-04,1000,0.095238,1.924431,-0.081561,-0.428572,-0.298542,-0.246,0.089400,0.089507
1,1974-05,1000,-0.130435,2.016424,0.048045,-0.222223,-0.297521,-0.246,0.089400,0.089507
2,1974-06,1000,0.050000,1.877682,0.187806,-0.080001,-0.296501,-0.246,0.089400,0.089507
3,1974-07,1000,-0.142857,1.927492,0.139016,-0.166668,-0.295198,-0.246,0.089400,0.089507
4,1974-08,1000,0.166667,1.766826,0.293167,-0.322582,-0.301428,-0.246,0.089400,0.089507
...,...,...,...,...,...,...,...,...,...,...
1455298,2020-09,317264,-0.050948,6.063467,0.673023,-0.178057,-0.078937,-115.174,-0.030312,-0.065911
1455299,2020-10,317264,0.023720,6.010200,0.725315,-0.185330,-0.079973,-115.174,-0.030312,-0.065911
1455300,2020-11,317264,0.332927,6.027104,0.701871,-0.356627,-0.086573,-115.174,-0.030312,-0.065911
1455301,2020-12,317264,0.115279,6.307898,0.414494,-0.428572,-0.093217,-115.174,-0.030312,-0.065911


In [149]:
m2_result = m2_data.groupby('Date').apply(cross_section1)
r2 = m2_data.groupby('Date').apply(cross_section2)
obs = m2_data.groupby('Date')['GVKEY'].count()
m2_result['adj_R2'] = r2
m2_result['Number_of_Firms'] = obs
m2_result.head(2)

,const,LogSize_-1,LogB/M_-1,"Return_-2,-12","LogIssues_-1,-36",Accruals_Yr-1,ROA_Yr-1,LogAG_Yr-1,adj_R2,Number_of_Firms
Date,,,,,,,,,,
1969-09,-0.048675,-0.008152,-0.069288,0.018891,0.011621,-0.000750,0.604530,0.040377,-0.019947,32
1969-10,0.098726,-0.019197,-0.070382,0.028369,0.011166,-0.001144,-0.313717,0.041936,-0.079415,33


In [150]:
m2_result.tail(20)

,const,LogSize_-1,LogB/M_-1,"Return_-2,-12","LogIssues_-1,-36",Accruals_Yr-1,ROA_Yr-1,LogAG_Yr-1,adj_R2,Number_of_Firms
Date,,,,,,,,,,
2019-08,-0.038108,-0.003056,-0.011633,0.042061,-0.010557,-8.738292e-06,0.001776,-0.020377,0.019377,2238
2019-09,0.020724,0.003928,0.023133,-0.061665,-0.025586,4.477874e-06,0.115217,-0.015797,0.107386,2222
2019-10,-0.024871,0.005969,0.000680,0.029154,-0.003850,1.244661e-05,-0.043082,-0.006038,0.011132,2181
2019-11,-0.023209,0.007495,-0.011787,-0.045643,-0.009626,1.533836e-05,-0.158941,0.023251,0.029838,2188
2019-12,0.014245,0.007614,0.030181,-0.006823,0.008050,-5.813007e-06,-0.098490,-0.006528,0.032423,2175
2020-01,0.039930,-0.015213,-0.033812,0.018844,0.006028,-9.869818e-06,0.013898,-0.008437,0.049973,2154
2020-02,-0.020691,-0.009133,-0.007296,-0.021123,-0.032149,-3.796644e-06,-0.109262,0.056489,0.012921,2160
2020-03,-0.277499,0.004159,-0.039177,0.021349,0.038635,2.967986e-05,-0.008433,-0.023924,0.051524,2148
2020-04,0.236999,-0.009323,0.013630,-0.007664,-0.059071,-2.732032e-05,-0.271140,0.172985,0.052778,2163


### Model 3

In [151]:
m3_data = import_data('GPEX1set3.csv')
m3_data

,Date,GVKEY,Return,LogSize_-1,LogB/M_-1,"Return_-2,-12","LogIssues_-1,-36",Accruals_Yr-1,ROA_Yr-1,LogAG_Yr-1,"DY_-1,-12","LogReturn_-13,-36","LogIssues_-1,-12","Turnover_-1,-12",Debt/Price_Yr-1,Sales/Price_Yr-1
0,1974-04,1000,0.095238,1.924431,-0.081561,-0.428572,-0.298542,-0.246,0.089400,0.089507,0.0,-0.551282,-0.153494,9739.004887,1.021711,5.509943
1,1974-05,1000,-0.130435,2.016424,0.048045,-0.222223,-0.297521,-0.246,0.089400,0.089507,0.0,-0.686046,-0.150169,9758.016991,0.931915,5.025683
2,1974-06,1000,0.050000,1.877682,0.187806,-0.080001,-0.296501,-0.246,0.089400,0.089507,0.0,-0.637681,-0.146841,9405.625892,1.070609,5.773643
3,1974-07,1000,-0.142857,1.927492,0.139016,-0.166668,-0.295198,-0.246,0.089400,0.089507,0.0,-0.636363,-0.132863,9020.064092,1.018589,5.493106
4,1974-08,1000,0.166667,1.766826,0.293167,-0.322582,-0.301428,-0.246,0.089400,0.089507,0.0,-0.367346,-0.126248,8754.809368,1.196121,6.450510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357661,2020-08,317264,-0.011710,6.076221,0.661244,-0.154099,-0.077902,-115.174,-0.030312,-0.065911,0.0,0.270834,-0.071939,252262.069994,1.598806,0.368673
1357662,2020-09,317264,-0.050948,6.063467,0.673023,-0.178057,-0.078937,-115.174,-0.030312,-0.065911,0.0,0.514577,-0.070355,252022.588874,1.619327,0.373405
1357663,2020-10,317264,0.023720,6.010200,0.725315,-0.185330,-0.079973,-115.174,-0.030312,-0.065911,0.0,0.519062,-0.067314,257688.600552,1.707922,0.393834
1357664,2020-11,317264,0.332927,6.027104,0.701871,-0.356627,-0.086573,-115.174,-0.030312,-0.065911,0.0,0.741259,-0.069822,252706.685796,1.679295,0.387233


In [152]:
m3_result = m3_data.groupby('Date').apply(cross_section1)
r2 = m3_data.groupby('Date').apply(cross_section2)
obs = m3_data.groupby('Date')['GVKEY'].count()
m3_result['adj_R2'] = r2
m3_result['Number_of_Firms'] = obs
m3_result.head(2)

,const,LogSize_-1,LogB/M_-1,"Return_-2,-12","LogIssues_-1,-36",Accruals_Yr-1,ROA_Yr-1,LogAG_Yr-1,"DY_-1,-12","LogReturn_-13,-36","LogIssues_-1,-12","Turnover_-1,-12",Debt/Price_Yr-1,Sales/Price_Yr-1,adj_R2,Number_of_Firms
Date,,,,,,,,,,,,,,,,
1970-01,-0.00176,-0.015496,0.009734,-0.162988,0.040739,-0.000302,-1.679048,0.069826,1.769051,0.050180,-0.043758,-0.000002,0.016398,-0.043461,0.424115,30
1970-02,-0.15158,0.007237,-0.044959,-0.085257,0.042866,-0.001551,-0.958331,0.035893,0.705110,0.068943,-0.452827,0.000004,0.003314,-0.002528,0.327306,32


In [153]:
m3_result.tail(20)

,const,LogSize_-1,LogB/M_-1,"Return_-2,-12","LogIssues_-1,-36",Accruals_Yr-1,ROA_Yr-1,LogAG_Yr-1,"DY_-1,-12","LogReturn_-13,-36","LogIssues_-1,-12","Turnover_-1,-12",Debt/Price_Yr-1,Sales/Price_Yr-1,adj_R2,Number_of_Firms
Date,,,,,,,,,,,,,,,,
2019-08,-0.026077,-0.001105,-0.016201,0.016719,-0.018708,-0.000007,-0.017979,0.008421,-0.064278,-0.021915,-0.008952,-8.870083e-08,-0.008829,-0.000960,0.030507,2084
2019-09,0.017436,0.001711,0.009929,-0.066756,-0.034071,0.000009,0.091569,-0.005186,0.424443,-0.013981,-0.040349,2.423544e-08,0.002316,0.004073,0.133473,2060
2019-10,-0.031521,0.005931,0.010026,0.044033,0.013546,0.000008,-0.060105,-0.029419,-0.087498,0.030401,-0.071928,2.705097e-08,-0.006677,0.003292,0.030128,2039
2019-11,-0.009790,0.005450,-0.007184,-0.034318,-0.003257,0.000009,-0.134051,0.026054,-0.274361,0.008453,-0.016728,3.927101e-08,-0.001249,-0.000844,0.032964,2031
2019-12,0.042754,0.003701,0.031870,-0.006255,-0.026794,-0.000007,-0.085119,-0.008555,-0.121144,-0.002449,0.075486,5.867450e-08,0.010921,-0.007308,0.039457,2018
2020-01,0.054868,-0.011588,-0.026155,0.013172,0.007758,-0.000011,-0.010959,-0.012424,-0.275433,0.002134,-0.012574,-1.317629e-07,-0.017242,-0.000449,0.061992,2006
2020-02,-0.013502,-0.011005,-0.007126,-0.005188,0.001564,-0.000011,-0.039187,0.015483,-0.171546,0.010377,-0.016599,4.996392e-08,-0.009741,-0.001213,0.013124,2004
2020-03,-0.240332,0.011097,-0.021928,-0.008646,0.028043,0.000022,-0.035173,-0.010573,-0.523013,-0.000672,0.065230,-2.690584e-07,-0.048845,0.000035,0.103305,1967
2020-04,0.167073,-0.009077,0.007770,0.030060,0.011475,-0.000019,-0.112447,0.067116,-0.228332,-0.007711,0.005799,3.238332e-07,0.014184,0.002158,0.074427,1913


### Part 2 compute the time series averages of the slope estimates and their standard errors. Calculate t-test and determine if the slope average is significantly different from zero.

In [154]:
def Fama_MacBeth_procedure(result,model):
    mean = result.iloc[:,:-2].mean() # mean of risk premium
    standard_error = np.sqrt(((result.iloc[:,:-2] - mean)**2).sum() /(len(result)-1)) / np.sqrt(len(result)) # standard error of risk premium
    t = mean / (standard_error) # t- stast
    p = t.apply( lambda x : scipy.stats.t.sf(abs(x), len(result)-1)*2) #p -value survive function
    star = p.apply(lambda x: '***' if x<=0.01 else '**' if x<=0.05 else '*' if x<=0.1 else ' ') # significant
    mean = mean.astype(str) +star
    df = []
    for i in t.index:
        row = []
        row.append(mean.loc[i])
        row.append(t.loc[i])
        df.append(pd.DataFrame(row,index=[i,i],columns=[model]))
    return pd.concat(df)

In [155]:
def change_index(df): 
    df= df.reset_index()
    df['index'] = 'Average of time series of ' + df['index']
    df['index'].iloc[2:] = df['index'].iloc[2:] + 'coefficient'
    for i in range(len(df)):
        if i %2 !=0:
            df['index'].iloc[i] = ' '
    df = df.set_index('index')
    return df

### Model 1

In [156]:
m1 = Fama_MacBeth_procedure(m1_result,'Model1')
m1= change_index(m1)
m1

,Model1
index,
Average of time series of const,0.01779453220677411***
,5.582568
Average of time series of LogSize_-1coefficient,-0.0010863287970862816***
,-2.93076
Average of time series of LogB/M_-1coefficient,0.0047228332900648125***
,7.565198
"Average of time series of Return_-2,-12coefficient",0.009976286172284984***
,6.626166


### Model 2

In [157]:
m2 = Fama_MacBeth_procedure(m2_result,'Model2')
m2 = change_index(m2)
m2

,Model2
index,
Average of time series of const,0.01796233193881151***
,5.608896
Average of time series of LogSize_-1coefficient,-0.001018803099537398***
,-2.888232
Average of time series of LogB/M_-1coefficient,0.003915994982874923***
,6.236552
"Average of time series of Return_-2,-12coefficient",0.008598530908881142***
,5.820084
"Average of time series of LogIssues_-1,-36coefficient",-0.0019772416307174768*


### Model 3

In [158]:
m3 = Fama_MacBeth_procedure(m3_result,'Model3')
m3 = change_index(m3)
m3

,Model3
index,
Average of time series of const,0.016641589983287543***
,5.690504
Average of time series of LogSize_-1coefficient,-0.0008658457162023558***
,-2.665991
Average of time series of LogB/M_-1coefficient,0.0029279855404058907***
,5.934508
"Average of time series of Return_-2,-12coefficient",0.007302413282462412***
,5.68447
"Average of time series of LogIssues_-1,-36coefficient",-0.0012403597523624947


In [159]:
model_results = pd.concat([m1,m2,m3]) 
model_results

,Model1,Model2,Model3
index,,,
Average of time series of const,0.01779453220677411***,NaN,NaN
,5.582568,NaN,NaN
Average of time series of LogSize_-1coefficient,-0.0010863287970862816***,NaN,NaN
,-2.93076,NaN,NaN
Average of time series of LogB/M_-1coefficient,0.0047228332900648125***,NaN,NaN
,7.565198,NaN,NaN
"Average of time series of Return_-2,-12coefficient",0.009976286172284984***,NaN,NaN
,6.626166,NaN,NaN
Average of time series of const,NaN,0.01796233193881151***,NaN


### Part3 Forecast 10 years rolling windows using Model 3

In [160]:
m3_data.set_index('Date',inplace=True)
forcast_period = m3_result.index[120:]
factor_name = m3_result.columns[1:-2]
predict_result = []
for i in range(120,len(m3_result)):
    risk_premium_estimators = m3_result.iloc[i-120:i].mean() # ten years risk premium estimators
    forest_month = m3_result.index[i-1]
    # actual result
    actual = m3_data.loc[m3_result.index[i],['GVKEY', 'Return']]
    actual = actual.rename(columns={'Return':'Actual Return'})
    actual = actual.reset_index()
    #predict
    factors = m3_data.loc[forest_month,factor_name]
    predict = (factors * risk_premium_estimators[1:-2]).sum(axis=1) + risk_premium_estimators[0]
    predict.name = 'Forecast/Predicted Return'
    predict_df = m3_data.loc[forest_month ,['GVKEY']]
    predict_df['Forecast/Predicted Return'] = predict
    predict_df = predict_df.reset_index(drop=True)
    # merge actual and predict results
    df = pd.merge(actual, predict_df, how='outer', on = 'GVKEY').dropna()
    df.Date = df.Date.ffill()
    df = df.set_index('Date')
    predict_result.append(df)

In [161]:
# final result
q3 =pd.concat(predict_result)
q3 = q3.reset_index()
q3['Date'] = pd.to_datetime(q3['Date']).apply(lambda x: x + timedelta(calendar.monthrange(x.year, x.month)[1]-1))
q3 = q3.set_index('Date')
q3.dropna()

,GVKEY,Actual Return,Forecast/Predicted Return
Date,,,
1980-01-31,1004,0.226695,0.011401
1980-01-31,1010,0.211679,0.007148
1980-01-31,1020,-0.035714,0.011717
1980-01-31,1025,0.201539,0.018524
1980-01-31,1040,-0.008547,0.006134
...,...,...,...
2021-03-31,264265,-0.046658,0.009738
2021-03-31,264416,0.251401,0.011641
2021-03-31,268208,0.110647,0.014565


### Part 4 Simple backtest: the annualized trading return rate averaging across all the stocks traded.

If the forecast/predicted return is positive (negative), buy (sell) 1 dollar of that stock. The maximum position of each stock is 1dollar, monthly rebalancing.

In [162]:
backtest = q3.copy()
backtest['signal'] = np.sign(backtest['Forecast/Predicted Return'])
backtest['strategy'] = backtest['Actual Return'] * backtest['signal']

In [163]:
backtest

,GVKEY,Actual Return,Forecast/Predicted Return,signal,strategy
Date,,,,,
1980-01-31,1004,0.226695,0.011401,1.0,0.226695
1980-01-31,1010,0.211679,0.007148,1.0,0.211679
1980-01-31,1020,-0.035714,0.011717,1.0,-0.035714
1980-01-31,1025,0.201539,0.018524,1.0,0.201539
1980-01-31,1040,-0.008547,0.006134,1.0,-0.008547
...,...,...,...,...,...
2021-03-31,264265,-0.046658,0.009738,1.0,-0.046658
2021-03-31,264416,0.251401,0.011641,1.0,0.251401
2021-03-31,268208,0.110647,0.014565,1.0,0.110647


In [164]:
month_return  = backtest.groupby(backtest.index)['strategy'].mean()
average_monthly_return = month_return.mean() *12
average_monthly_return

0.1425788611705671

### Part 5 Accuracy of Forecasting

In [165]:
def accuracy_rate(df):
    df['accuracy'] = np.sign(df['Actual Return'] * df['Forecast/Predicted Return'])
    return len(df[df['accuracy'] == 1]) / len(df)

In [166]:
ar = accuracy_rate(q3)
ar

0.5068149535068917

### Part 6 Forecast with Significant Factors

In [167]:
mean = m3_result.iloc[:,:-2].mean()
standard_error = m3_result.iloc[:,:-2].std() /np.sqrt(len(m3_result))
t = mean / (standard_error)
p = t.apply( lambda x : scipy.stats.t.sf(abs(x), len(m3_result)-1)*2) #p -value survive function
star = p.apply(lambda x: '***' if x<=0.01 else '**' if x<=0.05 else '*' if x<=0.1 else ' ') 

In [168]:
# select significant factors with 3 stars
m3_summary = pd.concat([mean,t,star],axis=1)
significant_factors = m3_summary[ (m3_summary.iloc[:,-1] == '***')].index.to_list()
significant_factors

['const',
 'LogSize_-1',
 'LogB/M_-1',
 'Return_-2,-12',
 'ROA_Yr-1',
 'LogAG_Yr-1',
 'Sales/Price_Yr-1']

In [169]:
m3_data_1=pd.concat([m3_data[['GVKEY', 'Return']],m3_data[significant_factors[1:]]],axis=1)
m3_data_1 =m3_data_1.reset_index()

In [170]:
m3_result_1 = m3_data_1.groupby('Date').apply(cross_section1)
r2 = m3_data_1.groupby('Date').apply(cross_section2)
obs = m3_data_1.groupby('Date')['GVKEY'].count()
m3_result_1['adj_R2'] = r2
m3_result_1['Number_of_Firms'] = obs
m3_result_1

,const,LogSize_-1,LogB/M_-1,"Return_-2,-12",ROA_Yr-1,LogAG_Yr-1,Sales/Price_Yr-1,adj_R2,Number_of_Firms
Date,,,,,,,,,
1970-01,0.195150,-0.028631,0.056319,-0.092661,-1.501916,0.026589,-0.042868,0.137934,30
1970-02,-0.027284,0.029982,-0.009615,0.014043,-0.636153,-0.071485,0.000656,0.057069,32
1970-03,0.013630,-0.003754,0.038585,-0.030060,0.419366,-0.000243,-0.007992,-0.037403,60
1970-04,-0.241807,0.025725,0.063234,-0.012937,0.363006,-0.124203,-0.004719,0.320566,89
1970-05,-0.121944,0.005050,0.013000,-0.049072,0.387131,-0.041722,-0.001069,-0.029250,87
...,...,...,...,...,...,...,...,...,...
2020-11,0.224904,-0.006028,0.020973,-0.066805,-0.111614,0.005063,0.012204,0.077559,1816
2020-12,0.180718,-0.013926,-0.007540,-0.001633,-0.034032,-0.031334,-0.001393,0.024681,1833
2021-01,0.198801,-0.020860,0.005135,0.062237,-0.077143,-0.027253,0.003891,0.128245,589


In [171]:
# forecast
m3_data_1 =m3_data_1.set_index('Date')
forcast_period = m3_result_1.index[120:]
factor_name = m3_result_1.columns[1:-2]
predict_result = []
for i in range(120,len(m3_result_1)):
    risk_premium_estimators = m3_result_1.iloc[i-120:i].mean() # ten years risk premium estimators
    forest_month = m3_result_1.index[i-1]
    # actual result
    actual = m3_data.loc[m3_result_1.index[i],['GVKEY', 'Return']]
    actual = actual.rename(columns={'Return':'Actual Return'})
    actual = actual.reset_index()
    #predict
    factors = m3_data_1.loc[forest_month,factor_name]
    predict = (factors * risk_premium_estimators[1:-2]).sum(axis=1) + risk_premium_estimators[0]
    predict.name = 'Forecast/Predicted Return'
    predict_df = m3_data_1.loc[forest_month ,['GVKEY']]
    predict_df['Forecast/Predicted Return'] = predict
    predict_df = predict_df.reset_index(drop=True)
    # merge actual and predict results
    df = pd.merge(actual, predict_df, how='outer', on = 'GVKEY').dropna()
    df.Date = df.Date.ffill()
    df = df.set_index('Date')
    predict_result.append(df)

In [172]:
im =pd.concat(predict_result)
im = im1.reset_index()
im['Date'] = pd.to_datetime(im['Date']).apply(lambda x: x + timedelta(calendar.monthrange(x.year, x.month)[1]-1))
im = im.set_index('Date')
im

,GVKEY,Actual Return,Forecast/Predicted Return,accuracy
Date,,,,
1980-03-01,1004,0.226695,0.012467,1.0
1980-03-01,1010,0.211679,0.009143,1.0
1980-03-01,1020,-0.035714,0.002061,-1.0
1980-03-01,1025,0.201539,0.021818,1.0
1980-03-01,1040,-0.008547,0.009595,-1.0
...,...,...,...,...
2021-04-30,264265,-0.046658,0.007669,-1.0
2021-04-30,264416,0.251401,0.011631,1.0
2021-04-30,268208,0.110647,0.016158,1.0


In [173]:
# backtest
backtest_im = im.copy()
backtest_im['signal'] = np.sign(backtest_im['Forecast/Predicted Return'])
backtest_im['strategy'] =backtest_im['Actual Return'] * backtest_im['signal']
month_return_im  = backtest_im.groupby(backtest_im.index)['strategy'].mean()
average_monthly_return_im = month_return_im.mean() *12
average_monthly_return_im

0.1471331413307806

In [174]:
# accuracy rate
ar_im = accuracy_rate(im)
ar_im

0.5072779235749816

### Part 7 Comparison

In [175]:
summary = pd.DataFrame({'Annualized trading return rate':[average_monthly_return,average_monthly_return_im],'Accuracy rate':[ar, ar_im]},index=['All Factors','Significant Factors']).T
summary['Diff'] = summary['Significant Factors'] - summary['All Factors'] 
summary = summary.T
summary

,Annualized trading return rate,Accuracy rate
All Factors,0.142579,0.506815
Significant Factors,0.147133,0.507278
Diff,0.004554,0.000463
